# Task 13.5: Multi-Agent Content Generation System

**Module:** 13 - AI Agents & Agentic Systems  
**Time:** 3 hours  
**Difficulty:** ⭐⭐⭐⭐ (Advanced)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand multi-agent architectures and when to use them
- [ ] Build a 3-agent content generation team
- [ ] Implement agent communication patterns
- [ ] Handle task delegation and collaboration
- [ ] Manage agent state and handoffs

---

## 📚 Prerequisites

- Completed: Tasks 13.1-13.4
- Knowledge of: Agent basics, prompting, workflows

---

## 🌍 Real-World Context

**Why multiple agents instead of one?**

Think of a software company:
- One person can't be an expert at everything
- Specialists collaborate: designers, developers, QA, PMs
- Each focuses on what they do best

**Multi-agent systems work the same way:**
- 🔬 **Researcher**: Finds and synthesizes information
- ✍️ **Writer**: Creates compelling content
- ✅ **Editor**: Reviews and improves quality

**Real applications:**
- 📰 **News Agencies**: Research → Write → Edit → Publish
- 💼 **Consulting**: Analyze → Strategize → Review → Present
- 🎮 **Game Dev**: Design → Implement → Test → Polish

---

## 🧒 ELI5: Multi-Agent Systems

> **Imagine a relay race...** 🏃
>
> One runner can't win alone. You need:
> - A **sprinter** who's fast at the start
> - A **steady runner** who maintains pace
> - A **finisher** who brings it home
>
> Each runner:
> - Has a specific skill
> - Knows when to receive the baton
> - Knows when to pass it on
>
> **Multi-agent AI is the same!**
> - Each agent has a role (researcher, writer, editor)
> - They pass work to each other
> - Together they accomplish more than one agent could
>
> ```
>   ┌──────────────┐    ┌──────────────┐    ┌──────────────┐
>   │  RESEARCHER  │───►│    WRITER    │───►│    EDITOR    │
>   │              │    │              │    │              │
>   │ "Here's what │    │ "Here's my   │    │ "Here's the  │
>   │  I found..." │    │  draft..."   │    │  final..."   │
>   └──────────────┘    └──────────────┘    └──────────────┘
>   ```

---

## Part 1: Environment Setup

In [ ]:
# Standard imports
import os
import sys
from pathlib import Path
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, field
from datetime import datetime
import json
import time

# LangChain imports
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate

print("Imports successful!")

In [ ]:
# Model Configuration - Change this if using a different model
LLM_MODEL = "llama3.1:8b"  # Options: "llama3.1:8b", "llama3.1:70b", "mistral:7b"

# Initialize LLM with timeout for long multi-agent operations
llm = Ollama(
    model=LLM_MODEL,
    temperature=0.7,
    request_timeout=120.0,  # 2 minute timeout
    base_url="http://localhost:11434"
)

print(f"✅ LLM initialized: {LLM_MODEL}")

---

## Part 2: Agent Architecture

Let's build a framework for our multi-agent system.

In [ ]:
# Define our Agent base class
@dataclass
class Agent:
    """Base class for all agents in the system."""
    name: str
    role: str
    personality: str
    llm: Any
    
    def get_system_prompt(self) -> str:
        """Generate the system prompt for this agent."""
        return f"""You are {self.name}, a {self.role}.

Personality: {self.personality}

Always:
- Stay in character as {self.name}
- Focus on your specific expertise
- Be concise and professional
- Clearly state your output"""
    
    def execute(self, task: str, context: str = "") -> str:
        """Execute a task and return the result."""
        prompt = f"""{self.get_system_prompt()}

Context from previous agents:
{context}

Your task:
{task}

Your response:"""
        
        print(f"\n🤖 {self.name} ({self.role}) is working...")
        response = self.llm.invoke(prompt)
        print(f"   ✓ {self.name} completed their task")
        
        return response

print("Agent class defined!")

In [ ]:
# Define a message for agent communication
@dataclass
class Message:
    """A message passed between agents."""
    sender: str
    recipient: str
    content: str
    timestamp: datetime = field(default_factory=datetime.now)
    
    def __str__(self):
        return f"[{self.sender} → {self.recipient}]: {self.content[:100]}..."

print("Message class defined!")

---

## Part 3: Creating Our Agent Team

Let's create three specialized agents for content creation.

In [ ]:
# Agent 1: The Researcher
researcher = Agent(
    name="Alex",
    role="Research Specialist",
    personality="""Thorough, detail-oriented, and curious. 
You love digging deep into topics and finding interesting facts. 
You organize information clearly with bullet points and sections.
You always cite your reasoning.""",
    llm=llm
)

print(f"Created: {researcher.name} - {researcher.role}")

In [ ]:
# Agent 2: The Writer
writer = Agent(
    name="Sam",
    role="Content Writer",
    personality="""Creative, engaging, and articulate.
You turn dry facts into compelling stories.
You write with a clear structure: hook, body, conclusion.
You use vivid examples and analogies.""",
    llm=llm
)

print(f"Created: {writer.name} - {writer.role}")

In [ ]:
# Agent 3: The Editor
editor = Agent(
    name="Jordan",
    role="Editorial Quality Reviewer",
    personality="""Meticulous, constructive, and quality-focused.
You catch errors others miss.
You improve clarity without changing the voice.
You provide specific, actionable feedback.""",
    llm=llm
)

print(f"Created: {editor.name} - {editor.role}")

In [ ]:
# Summary of our team
team = [researcher, writer, editor]

print("\n" + "="*60)
print("CONTENT CREATION TEAM")
print("="*60)

for agent in team:
    print(f"\n{agent.name} ({agent.role})")
    print(f"  {agent.personality.split(chr(10))[0]}")

---

## Part 4: The Orchestrator

The orchestrator manages the workflow and communication between agents.

In [ ]:
@dataclass
class ContentTask:
    """A content creation task."""
    topic: str
    content_type: str  # blog, article, social
    target_audience: str
    tone: str  # professional, casual, technical
    
    # Results from each stage
    research: str = ""
    draft: str = ""
    final: str = ""
    
    # Metadata
    messages: List[Message] = field(default_factory=list)
    status: str = "pending"

print("ContentTask class defined!")

In [ ]:
class ContentCreationOrchestrator:
    """Orchestrates the multi-agent content creation workflow."""
    
    def __init__(self, researcher: Agent, writer: Agent, editor: Agent):
        self.researcher = researcher
        self.writer = writer
        self.editor = editor
        self.history: List[Message] = []
    
    def log_message(self, sender: str, recipient: str, content: str):
        """Log a message between agents."""
        msg = Message(sender, recipient, content)
        self.history.append(msg)
    
    def run_research_phase(self, task: ContentTask) -> str:
        """Have the researcher gather information."""
        research_task = f"""Research the topic: {task.topic}

Target audience: {task.target_audience}
Content type: {task.content_type}
Tone: {task.tone}

Provide:
1. Key facts and statistics
2. Main points to cover
3. Interesting angles or hooks
4. Potential challenges or counterarguments

Format your research clearly with sections."""
        
        research = self.researcher.execute(research_task)
        task.research = research
        
        self.log_message(self.researcher.name, self.writer.name, research)
        return research
    
    def run_writing_phase(self, task: ContentTask) -> str:
        """Have the writer create the content."""
        writing_task = f"""Write a {task.content_type} about: {task.topic}

Target audience: {task.target_audience}
Tone: {task.tone}

Use the research provided to create engaging content.
Include a compelling hook, clear structure, and strong conclusion."""
        
        context = f"Research from {self.researcher.name}:\n{task.research}"
        draft = self.writer.execute(writing_task, context)
        task.draft = draft
        
        self.log_message(self.writer.name, self.editor.name, draft)
        return draft
    
    def run_editing_phase(self, task: ContentTask) -> str:
        """Have the editor review and improve."""
        editing_task = f"""Review and improve this {task.content_type}.

Target audience: {task.target_audience}
Desired tone: {task.tone}

Tasks:
1. Fix any grammatical or spelling errors
2. Improve clarity and flow
3. Ensure the tone matches the audience
4. Strengthen weak sections
5. Polish the opening and closing

Provide the FINAL polished version."""
        
        context = f"Draft from {self.writer.name}:\n{task.draft}"
        final = self.editor.execute(editing_task, context)
        task.final = final
        
        self.log_message(self.editor.name, "OUTPUT", final)
        return final
    
    def create_content(self, task: ContentTask) -> ContentTask:
        """Run the full content creation workflow."""
        print("\n" + "="*60)
        print(f"📝 CREATING: {task.content_type.upper()} about '{task.topic}'")
        print("="*60)
        
        task.status = "in_progress"
        start_time = time.time()
        
        # Phase 1: Research
        print("\n--- Phase 1: Research ---")
        self.run_research_phase(task)
        
        # Phase 2: Writing
        print("\n--- Phase 2: Writing ---")
        self.run_writing_phase(task)
        
        # Phase 3: Editing
        print("\n--- Phase 3: Editing ---")
        self.run_editing_phase(task)
        
        elapsed = time.time() - start_time
        task.status = "completed"
        
        print(f"\n✅ Content creation completed in {elapsed:.1f} seconds!")
        
        return task

print("ContentCreationOrchestrator defined!")

---

## Part 5: Running the Multi-Agent System

In [ ]:
# Create the orchestrator
orchestrator = ContentCreationOrchestrator(
    researcher=researcher,
    writer=writer,
    editor=editor
)

print("Orchestrator created!")

In [ ]:
# Create a content task
task1 = ContentTask(
    topic="How AI Agents are Transforming Software Development",
    content_type="blog post",
    target_audience="Software developers interested in AI",
    tone="professional but accessible"
)

print(f"Task created: {task1.topic}")
print(f"Type: {task1.content_type}")
print(f"Audience: {task1.target_audience}")

In [ ]:
# Run the multi-agent workflow
completed_task = orchestrator.create_content(task1)

In [ ]:
# View the results
print("\n" + "="*60)
print("📊 RESEARCH OUTPUT")
print("="*60)
print(completed_task.research[:1500])
print("...")

In [ ]:
print("\n" + "="*60)
print("✍️ WRITER'S DRAFT")
print("="*60)
print(completed_task.draft[:1500])
print("...")

In [ ]:
print("\n" + "="*60)
print("✅ FINAL EDITED VERSION")
print("="*60)
print(completed_task.final)

In [ ]:
# View the communication history
print("\n" + "="*60)
print("💬 AGENT COMMUNICATION LOG")
print("="*60)

for msg in orchestrator.history:
    print(f"\n{msg.timestamp.strftime('%H:%M:%S')} | {msg.sender} → {msg.recipient}")
    print(f"   {msg.content[:100]}...")

---

## Part 6: Advanced Pattern - Feedback Loops

Let's add the ability for agents to request revisions from each other.

In [ ]:
class AdvancedOrchestrator(ContentCreationOrchestrator):
    """Orchestrator with feedback loops between agents."""
    
    def __init__(self, researcher: Agent, writer: Agent, editor: Agent, max_revisions: int = 2):
        super().__init__(researcher, writer, editor)
        self.max_revisions = max_revisions
    
    def get_editor_feedback(self, task: ContentTask) -> tuple[bool, str]:
        """Get structured feedback from the editor."""
        feedback_task = f"""Review this {task.content_type} draft:

{task.draft}

Respond in this format:
DECISION: APPROVE or REVISE
FEEDBACK: [Your specific feedback]

Only approve if the content is publication-ready."""
        
        response = self.editor.execute(feedback_task)
        
        approved = "APPROVE" in response.upper()
        return approved, response
    
    def request_revision(self, task: ContentTask, feedback: str) -> str:
        """Ask the writer to revise based on feedback."""
        revision_task = f"""Revise your {task.content_type} based on editor feedback.

Your original draft:
{task.draft}

Editor's feedback:
{feedback}

Provide the revised version."""
        
        context = f"Editor feedback from {self.editor.name}"
        revised = self.writer.execute(revision_task, context)
        
        self.log_message(self.editor.name, self.writer.name, f"Revision requested: {feedback[:100]}...")
        self.log_message(self.writer.name, self.editor.name, f"Revision submitted: {revised[:100]}...")
        
        return revised
    
    def create_content_with_feedback(self, task: ContentTask) -> ContentTask:
        """Create content with revision loops."""
        print("\n" + "="*60)
        print(f"📝 CREATING (with feedback): {task.topic}")
        print("="*60)
        
        # Phase 1: Research
        print("\n--- Phase 1: Research ---")
        self.run_research_phase(task)
        
        # Phase 2: Writing
        print("\n--- Phase 2: Writing ---")
        self.run_writing_phase(task)
        
        # Phase 3: Review/Revision Loop
        revision_count = 0
        approved = False
        
        while not approved and revision_count < self.max_revisions:
            print(f"\n--- Phase 3: Review (attempt {revision_count + 1}) ---")
            
            approved, feedback = self.get_editor_feedback(task)
            
            if approved:
                print("   ✓ Draft approved!")
            else:
                print("   ⟳ Revision requested...")
                task.draft = self.request_revision(task, feedback)
                revision_count += 1
        
        # Phase 4: Final Polish
        print("\n--- Phase 4: Final Polish ---")
        self.run_editing_phase(task)
        
        print(f"\n✅ Completed with {revision_count} revision(s)!")
        task.status = "completed"
        
        return task

print("AdvancedOrchestrator with feedback loops defined!")

In [ ]:
# Test the advanced orchestrator
advanced_orchestrator = AdvancedOrchestrator(
    researcher=researcher,
    writer=writer,
    editor=editor,
    max_revisions=2
)

task2 = ContentTask(
    topic="Why DGX Spark is Perfect for AI Developers",
    content_type="social media post",
    target_audience="AI/ML developers on LinkedIn",
    tone="enthusiastic and informative"
)

result = advanced_orchestrator.create_content_with_feedback(task2)

In [ ]:
print("\n" + "="*60)
print("FINAL OUTPUT")
print("="*60)
print(result.final)

---

## ⚠️ Common Mistakes

### Mistake 1: Agents with Overlapping Roles

In [ ]:
# ❌ Wrong: Roles too similar
# agent1 = Agent("Writer", "Writes content")
# agent2 = Agent("Author", "Creates written content")  # Same thing!

# ✅ Right: Clear, distinct roles
# researcher - gathers facts
# writer - creates draft
# editor - reviews and polishes

print("Each agent should have a clear, unique responsibility.")

### Mistake 2: No Context Passing Between Agents

In [ ]:
# ❌ Wrong: Each agent works in isolation
# research = researcher.execute("Research AI")
# draft = writer.execute("Write about AI")  # Doesn't see research!

# ✅ Right: Pass context between agents
# research = researcher.execute("Research AI")
# draft = writer.execute("Write about AI", context=research)  # Uses research!

print("Always pass relevant context between agents!")

### Mistake 3: No Termination Condition

In [ ]:
# ❌ Wrong: Infinite revision loop
# while not editor_approves:
#     writer.revise()  # Could loop forever!

# ✅ Right: Always have a max iteration limit
max_revisions = 3
revision_count = 0

# while not approved and revision_count < max_revisions:
#     revise()
#     revision_count += 1

print("Always set maximum iterations for loops!")

---

## 🎉 Checkpoint

You've learned:
- ✅ Why multi-agent systems are powerful
- ✅ How to design agents with distinct roles
- ✅ Building an orchestrator to coordinate agents
- ✅ Implementing feedback loops between agents
- ✅ Managing communication and state

---

## 🚀 Challenge (Optional)

Extend the system with:
1. A **Fact Checker** agent that verifies claims
2. An **SEO Specialist** that optimizes for search
3. A **Translator** that creates multilingual versions

---

## 📖 Further Reading

- [CrewAI Documentation](https://docs.crewai.com/)
- [AutoGen Multi-Agent](https://microsoft.github.io/autogen/)
- [LangGraph Multi-Agent](https://langchain-ai.github.io/langgraph/tutorials/multi_agent/)

---

## 🧹 Cleanup

In [ ]:
# Comprehensive cleanup for DGX Spark
import gc

# Clear GPU memory if available
try:
    import torch
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
        allocated = torch.cuda.memory_allocated() / 1e9
        print(f"✅ GPU memory cleared ({allocated:.2f} GB still allocated)")
except ImportError:
    pass

# Python garbage collection
gc.collect()
print("✅ Cleanup complete!")

---

## 🎓 Summary

In this notebook, you built a complete multi-agent content creation system:

1. **Researcher Agent**: Gathers and organizes information
2. **Writer Agent**: Creates engaging content from research
3. **Editor Agent**: Reviews and polishes the final output
4. **Orchestrator**: Coordinates the workflow and communication

**Key patterns learned:**
- Sequential agent pipelines
- Context passing between agents
- Feedback loops for quality improvement
- Communication logging

**Next up:** Task 13.6 - Agent Benchmarking Framework